In [ ]:

# from models.pytorch.narrative_stack.stage1.preprocessing import generate_concept_unit_embeddings, generate_concepts_report
import os
from db import DbUsGaap
from pathlib import Path
from config import project_paths

db_us_gaap = DbUsGaap()

WEBSOCKET_ADDRESS = "127.0.0.1:7000"
# WEBSOCKET_ADDRESS = "192.168.0.10:3000"

# Rust data directory
csv_data_dir = Path(
    project_paths.rust_data / "truncated_csvs" # "june-us-gaap"  # Where CSV data is read from (once CSV file per symbol)
)


In [ ]:
# from simd_r_drive import DataStore
from simd_r_drive_ws_client import DataStoreWsClient

# client = DataStoreNetClient("127.0.0.1:50323")
from us_gaap_store import UsGaapStore

data_store = DataStoreWsClient(WEBSOCKET_ADDRESS)

us_gaap_store = UsGaapStore(data_store)

In [ ]:
# data_store.batch_write([
#     (b"testing", b"123"),
#     (b"testing2", b"1234"),
# ])

# data_store.batch_read([b"testing", b"testing2", b"testing3"])

In [ ]:
# Ingest

us_gaap_store.ingest_us_gaap_csvs(csv_data_dir, db_us_gaap)
us_gaap_store.generate_pca_embeddings()

In [ ]:
import numpy as np

# display(us_gaap_store.get_triplet_count())

# display(us_gaap_store.get_pair_count())

# # # for i in range(0, 10):
# # #     display(lookup_by_index(store, i * 20))

# display(us_gaap_store.batch_lookup_by_indices([1]))
display(us_gaap_store.lookup_by_index(80))


# Validate scaler (TODO: Make test to do random lookups)
# data = us_gaap_store.lookup_by_index(500000)
# transformed = data["scaler"].transform([[data["unscaled_value"]]])[0][0]
# assert np.isclose(transformed, data["scaled_value"]), (
#     f"Mismatch: transformed={transformed}, stored={data['scaled_value']}"
# )

In [ ]:
# stored_embedding_matrix, _pairs = us_gaap_store.get_embedding_matrix()

# stored_embedding_matrix

In [ ]:
# us_gaap_store.load_pca_model()

In [ ]:
# TODO: Move

# # NOTE: For debugging / monitoring purposes only
# # Determine "category stack" depths

# from collections import defaultdict
# import numpy as np
# from utils.csv import walk_us_gaap_csvs, get_filtered_us_gaap_form_rows_for_symbol

# class RunningStats:
#     def __init__(self):
#         self.count = 0
#         self.total = 0
#         self.max_val = 0
#         self.values = []

#     def update(self, val: int):
#         self.count += 1
#         self.total += val
#         self.max_val = max(self.max_val, val)
#         self.values.append(val)  # Optional: remove this if median not needed

#     def finalize(self):
#         result = {
#             "avg": self.total / self.count if self.count else 0,
#             "max": self.max_val,
#         }
#         if self.values:
#             result["median"] = float(np.median(self.values))
#         return result

# # Initialize running stats per key
# stats = defaultdict(RunningStats)

# gen = walk_us_gaap_csvs(data_dir, db_us_gaap, "row")

# try:
#     while True:
#         row = next(gen)
#         counter = defaultdict(int)
#         for entry in row.entries:
#             key = (entry.balance_type or "none", entry.period_type or "none")
#             counter[key] += 1
#         for key, val in counter.items():
#             stats[key].update(val)
# except StopIteration:
#     pass

# # Final summary
# summary = {key: stat.finalize() for key, stat in stats.items()}

# from pprint import pprint
# pprint(summary)


In [ ]:
# TODO: Move

# # NOTE: For debugging / monitoring purposes only

# import numpy as np
# from utils.csv import walk_us_gaap_csvs, UsGaapRowRecord
# from collections import defaultdict


# def generate_concepts_report_from_walker(
#     data_dir: Path,
#     db_us_gaap: DbUsGaap,
#     filtered_symbols: set[str] | None = None,
# ):
#     gen = walk_us_gaap_csvs(
#         data_dir=data_dir,
#         db_us_gaap=db_us_gaap,
#         walk_type="row",
#         filtered_symbols=filtered_symbols,
#     )

#     unit_stats = defaultdict(list)
#     concept_by_unit = defaultdict(set)

#     try:
#         while True:
#             row = next(gen)
#             if isinstance(row, UsGaapRowRecord):
#                 for entry in row.entries:
#                     unit_stats[entry.uom].append(entry.value)
#                     concept_by_unit[entry.uom].add(entry.concept)
#     except StopIteration as stop:
#         summary = stop.value

#     print(f"\n✅ Scanned {len(summary.csv_files)} files.")
#     print(
#         f"📦 Found {len(unit_stats)} numeric units and "
#         f"{len(summary.non_numeric_units)} non-numeric units."
#     )

#     for unit, values in sorted(unit_stats.items()):
#         arr = np.array(values)
#         print(f"🔹 {unit}")
#         print(f"   Count: {len(arr)}")
#         print(f"   Min:   {arr.min():,.4f}")
#         print(f"   Max:   {arr.max():,.4f}")
#         print(f"   Mean:  {arr.mean():,.4f}")
#         print(f"   Std:   {arr.std():,.4f}")
#         print(f"   Concepts: {', '.join(sorted(concept_by_unit[unit]))}")

#     if summary.non_numeric_units:
#         print("\n⚠️ Non-numeric units encountered:")
#         for unit in sorted(summary.non_numeric_units):
#             print(f"  - {unit}")

#     total_values = sum(len(v) for v in unit_stats.values())
#     print(f"\n🧮 Total values extracted: {total_values:,}")


# generate_concepts_report_from_walker(data_dir, db_us_gaap, None)


In [ ]:
from models.pytorch.narrative_stack.stage1.preprocessing.plots import (
    plot_pca_explanation,
)

embedding_matrix, _ = us_gaap_store.get_embedding_matrix()

# embedding_matrix.shape

dim = plot_pca_explanation(embedding_matrix, variance_threshold=0.95)

display(dim)

In [ ]:
# from utils.pytorch import get_device, seed_everything

# device = get_device()

# logging.info("Collecting concept unit pairs...")
# concept_unit_pairs = collect_concept_unit_pairs(extracted_concept_unit_value_data)

# logging.info(f"Total concept unit pairs: {len(concept_unit_pairs)}")

# logging.info("Generating concept unit embeddings...")
# concept_unit_embeddings = generate_concept_unit_embeddings(concept_unit_pairs, device=device)
# # concept_unit_embeddings_2 = generate_concept_unit_embeddings(concept_unit_pairs, device=device)



In [ ]:
# import numpy as np
# import torch
# from tqdm import tqdm

# # Normalize for cosine similarity
# A = concept_unit_embeddings_1
# B = concept_unit_embeddings_2

# A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
# B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)

# # Cosine similarity per row
# cos_sim = np.sum(A_norm * B_norm, axis=1)

# # Report
# print(f"Cosine similarity:")
# print(f"  Mean: {cos_sim.mean():.8f}")
# print(f"  Min:  {cos_sim.min():.8f}")
# print(f"  Std:  {cos_sim.std():.8f}")

# # Optional: show rows below threshold
# threshold = 0.999
# bad_indices = np.where(cos_sim < threshold)[0]
# print(f"\n🔻 Below {threshold}: {len(bad_indices)} / {len(cos_sim)} rows")
# if len(bad_indices):
#     for idx in bad_indices[:10]:
#         print(f"  Row {idx}: cosine = {cos_sim[idx]:.8f}")


In [ ]:
from models.pytorch.narrative_stack.stage1.preprocessing.plots import (
    plot_semantic_embeddings,
)

# plot_semantic_embeddings(stored_embedding_matrix, title="PCA Semantic Embedding Scatterplot")
plot_semantic_embeddings(embedding_matrix, title="Raw Semantic Embedding Scatterplot")